In [ ]:
from rag.GoogleSearchScrapper import GoogleSearchModule

In [ ]:
# Create an instance of GoogleSearchModule with the topic you want to search for
search_module = GoogleSearchModule("Write a post about the benefits of using a chatbot on a website.")

In [ ]:
# Get URLs, filtered to exclude certain social media sites
filtered_urls = search_module.get_url("Write a post about the benefits of using a chatbot on a website.")

In [ ]:
# Print the filtered URLs
print(filtered_urls)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
loaded_documents = []
for url in filtered_urls:
    loader = WebBaseLoader(url)
    loaded_documents += loader.load()

In [ ]:
loaded_documents

In [ ]:
pretty_print_docs(loaded_documents)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(loaded_documents)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
db = FAISS.from_documents(documents, embeddings)

In [ ]:
query = "Write a short post on the benefits of using a chatbot on a website."
retriever = db.as_retriever(search_kwargs={"k": 10})

In [ ]:
retriever.invoke(query)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

In [ ]:
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
compressed_docs = compression_retriever.invoke(query)

In [ ]:
compressed_docs

In [ ]:
pretty_print_docs(compressed_docs)

___

In [ ]:
import os
from dotenv import load_dotenv
from langchain.schema import HumanMessage
from camel_pipeline import camel_pipeline
from rag import return_filtered_urls, return_camel_response_doc, return_url_doc, ingest_all, retrieval_pipeline

In [ ]:
assistant_role_name = "Content Writer"
user_role_name = "Founder and CEO"
user_task = "Write a post about the benefits of using a chatbot on a website."

In [ ]:
specified_task, camel_response = camel_pipeline(assistant_role_name, user_role_name, user_task)

In [ ]:
specified_task

In [ ]:
camel_response

In [ ]:
urls = return_filtered_urls(specified_task)
urls

In [ ]:
camel_response_doc = return_camel_response_doc()

In [ ]:
url_doc = return_url_doc(urls)

In [ ]:
documents = camel_response_doc + url_doc

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
faiss_db = FAISS.from_documents(documents, embeddings)

In [ ]:
retriever = retrieval_pipeline(faiss_db)

In [ ]:
ranked_docs = retriever.invoke(specified_task)
pretty_print_docs(ranked_docs)

In [ ]:
from langchain.chains import RetrievalQA

___

In [1]:
### Setting up the CAMEL-RAG pipeline
# Importing Dependencies
import os
from dotenv import load_dotenv
from langchain.schema import HumanMessage

from camel_pipeline import camel_pipeline
from rag import (return_filtered_urls, return_camel_response_doc, return_url_doc, 
                ingest_all, retrieval_pipeline, pretty_print_docs, get_llm_chain,
                get_custom_prompt_template)

# Load the Environment Variables
load_dotenv()
# Set up the OpenAI API key
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "")

# Define the roles and task
assistant_role_name = "Content Writer"
user_role_name = "Founder and CEO"
user_task = "Write a linkedin post on brainstorming and visioning techniques."

# Run the CAMEL pipeline
specified_task, camel_response = camel_pipeline(assistant_role_name, user_role_name, user_task)
# Get the URLs for the specified task
urls = return_filtered_urls(specified_task)
# Get the CAMEL response document
camel_response_doc = return_camel_response_doc()
# Get the URL document
url_doc = return_url_doc(urls)
# Combine the CAMEL response document and URL document
docs = camel_response_doc + url_doc
# Injest all the documents from the web into FAISS vectorstore
db = ingest_all(docs)
# Create the retrieval pipeline
retriever = retrieval_pipeline(db)
# Retrieve the ranked documents
ranked_docs = retriever.invoke(specified_task)

2024-06-04 20:41:59,230 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:01,327 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:02,907 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:03,906 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:05,335 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:06,425 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:08,297 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:09,341 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 20:42:12,340 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [2]:
specified_task

'Write a LinkedIn post discussing the use of virtual reality technology in brainstorming and visioning techniques, highlighting its potential to enhance creativity and collaboration in remote work environments. Include examples of successful implementation and encourage professionals to explore this innovative approach.'

In [3]:
camel_response

"Virtual reality technology is revolutionizing the way we approach brainstorming and visioning techniques in the remote work environment. As professionals navigate the challenges of collaborating from different locations, VR offers an innovative solution to enhance creativity and foster effective teamwork. This post will explore the potential of VR in transforming the way we ideate and visualize, ultimately leading to more impactful outcomes.One notable example of successful implementation of virtual reality technology in brainstorming and visioning techniques is seen in the automotive industry. Companies like Ford and Audi have utilized VR to create immersive virtual environments where teams can visualize and iterate on vehicle designs, leading to more efficient and collaborative ideation processes. This has resulted in accelerated innovation and improved design outcomes.Professionals across industries are encouraged to explore the innovative approach of using virtual reality technolo

In [4]:
ranked_docs

[Document(page_content='Virtual reality technology is revolutionizing the way we approach brainstorming and visioning techniques in the remote work environment. As professionals navigate the challenges of collaborating from different locations, VR offers an innovative solution to enhance creativity and foster effective teamwork. This post will explore the potential of VR in transforming the way we ideate and visualize, ultimately leading to more impactful outcomes.One notable example of successful implementation of virtual reality technology in brainstorming and visioning techniques is seen in the automotive industry. Companies like Ford and Audi have utilized VR to create immersive virtual environments where teams can visualize and iterate on vehicle designs, leading to more efficient and collaborative ideation processes. This has resulted in accelerated innovation and improved design outcomes.Professionals across industries are encouraged to explore the innovative approach of using v

In [5]:
context = ""
for doc in ranked_docs:
    context += doc.page_content + "\n\n"

In [6]:
# Get the custom prompt template
prompt = get_custom_prompt_template(assistant_role_name, user_role_name, context)
# Get the LLM chain
llm_chain = get_llm_chain(retriever, prompt)
# Get the response from the LLM chain
response = llm_chain(specified_task)
# Print the response
print(response)

NameError: name 'task_specified' is not defined

---

In [4]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [5]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [1]:
from rag import get_custom_prompt_template
from rag_pipeline import rag_pipeline_context, rag_pipeline_response

In [2]:
assistant_role_name = "Content Writer"
user_role_name = "Founder and CEO"
specified_task= 'Write a LinkedIn post discussing the use of virtual reality technology in brainstorming and visioning techniques, highlighting its potential to enhance creativity and collaboration in remote work environments. Include examples of successful implementation and encourage professionals to explore this innovative approach.'

In [3]:
context = rag_pipeline_context(assistant_role_name, user_role_name, specified_task)

2024-06-04 23:13:36,574 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-04 23:13:36,801 - INFO - Loading faiss with AVX2 support.
2024-06-04 23:13:36,821 - INFO - Successfully loaded faiss with AVX2 support.
2024-06-04 23:13:38,224 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [4]:
prompt = get_custom_prompt_template(assistant_role_name, user_role_name, context)
prompt

ChatPromptTemplate(input_variables=['specified_task'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a Content Writer. Your role is to provide assistance to Founder and CEO in completing the task. \n    You can refer to the context below to generate a response to the user\'s task and respond with your expertise.\n    Contex: Fostering innovation and problem-solving within a remote team environment presents unique challenges that demand unconventional brainstorming techniques.I\'m sorry, I cannot fulfill this request as I do not have personal anecdotes or case studies to share.In a hypothetical scenario, a remote team faced a complex problem that required innovative solutions. By using unconventional brainstorming techniques such as "reverse thinking" and "random word association" during a virtual brainstorming session, the team was able to generate out-of-the-box ideas and effectively solve the challenge, leading to increased producti